<a href="https://colab.research.google.com/github/DOMINION-JOHN1/voting_eligibility_model/blob/main/preprocessing_of_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Define the file path of the np.array document
file_path = '/content/drive/My Drive/.npy'

# Load the np.array document using np.load()
loaded_array = np.load(file_path)

In [ ]:
import tensorflow as tf

x=loaded_array


In [ ]:
eligibility=pd.read_csv('/content/drive/My Drive/eligibility.csv')

In [ ]:

eligibility['eligibility']=eligibility['eligibility'].replace('55', 'Eligible')
eligibility['eligibility']=eligibility['eligibility'].replace('35', 'Eligible')
eligibility['eligibility'] = eligibility['eligibility'].replace({'Eligible': 1, 'Not Eligible': 0})
target_label = np.array(eligibility['eligibility'])
target_label.astype('float32')
target_label.shape
y=target_label

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Print the sizes of the train and test sets
print("Training data size:", len(X_train))
print("Testing data size:", len(X_test))

Training data size: 10850
Testing data size: 2713


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPool2D, Dropout, Flatten, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(128, 128, 3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(Activation("relu"))
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 activation_3 (Activation)   (None, 126, 126, 32)      0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 activation_4 (Activation)   (None, 61, 61, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 59, 59, 64)        36928     
                                                      

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

# Define the file path for saving the best model
checkpoint_filepath = '/content/drive/My Drive/best_model.h5'

# Create callbacks
checkpointer = ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=5, min_lr=1e-6)

epochs = 100
batch_size = 15


# Fit the model with callbacks
hist = model.fit(X_train, y_train, batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=[checkpointer, early_stopping, reduce_lr],
    verbose=2
)


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.57390, saving model to /content/drive/My Drive/best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


724/724 - 26s - loss: 0.5956 - accuracy: 0.7358 - val_loss: 0.5739 - val_accuracy: 0.7346 - lr: 0.0010 - 26s/epoch - 36ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.57390 to 0.57209, saving model to /content/drive/My Drive/best_model.h5
724/724 - 10s - loss: 0.5818 - accuracy: 0.7359 - val_loss: 0.5721 - val_accuracy: 0.7346 - lr: 0.0010 - 10s/epoch - 13ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.57209
724/724 - 9s - loss: 0.5749 - accuracy: 0.7359 - val_loss: 0.5747 - val_accuracy: 0.7346 - lr: 0.0010 - 9s/epoch - 13ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.57209
724/724 - 9s - loss: 0.5682 - accuracy: 0.7359 - val_loss: 0.5725 - val_accuracy: 0.7346 - lr: 0.0010 - 9s/epoch - 13ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.57209
724/724 - 9s - loss: 0.5557 - accuracy: 0.7354 - val_loss: 0.5733 - val_accuracy: 0.7346 - lr: 0.0010 - 9s/epoch - 13ms/step
Epoch 6/100

Epoch 6: val_loss did not improve from 0.57209
724/724 - 9s

In [ ]:
# Save the entire model
model.save('eligibility_checker.h5')